In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
from pathlib import Path
from copy import deepcopy
from functools import partial

import numpy as np
import pandas as pd

from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark
from bellek.musique.qa import answer_question_cte

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
def perfect_retrieval(docs, query):
    return [doc for doc in docs if doc['is_supporting']]

In [5]:
N_RUNS = 3

In [6]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-common/base-dataset-validation.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
# df = df.sample(10)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [7]:
# with open("./cte-few-shot-examples.json") as f:
#     few_shot_examples = json.load(f)


# len(few_shot_examples)

In [8]:
selected_few_shot_examples = [
    {
        "id": "2hop__784447_126070",
        "context": 'Glenhis Hern\u00e1ndez (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World\nChampion in middleweight.\n\nThe current mayor of Havana ("President of the People\'s Power Provincial Assembly") is Marta Hern\u00e1ndez Romero, she\nwas elected on March 5, 2011.',
        "question": "Who is the current mayor of the city Glenhis Hern\u00e1ndez was born?",
        "answers": ["Marta Hern\u00e1ndez Romero"],
        "cte_generation": "Triplets: \nGlenhis Hern\u00e1ndez | birth place | Havana\nMarta Hern\u00e1ndez Romero | serves as | mayor of Havana\n\nAnswer: Marta Hern\u00e1ndez Romero",
    },
    {
        "id": "2hop__823584_776926",
        "context": '# Rotst\u00f6ckli\nThe Rotst\u00f6ckli (2,901 m) is a peak of the Urner Alps below the Titlis, on the border between the Swiss cantons of Obwalden and Nidwalden. It is Nidwalden\'s highest point. The summit is split between the municipalities of Engelberg (Obwalden) and Wolfenschiessen (Nidwalden).\n# Uri Alps\nThe Uri Alps (also known as "Urner Alps", ) are a mountain range in Central Switzerland and part of the Western Alps. They extend into the cantons of Obwalden, Valais, Bern, Uri and Nidwalden and are bordered by the Bernese Alps (Grimsel Pass) and the Emmental Alps to the west (the four lakes: Lungerersee, Sarnersee, Wichelsee, and Alpnachersee), the Schwyzer Alps to the north (Lake Lucerne), the Lepontine Alps to the south (the valley of Urseren with Andermatt) and the Glarus Alps to the east (Reuss).',
        "question": "What area contains the region that encompasses Rotst\u00f6ckli?",
        "cte_generation": "Triplets:\nRotst\u00f6ckli | part of | Urner Alps\nUrner Alps | part of | Western Alps\n\nAnswer: Western Alps",
        "cot_generation": "Reasoning:\n- The context indicates that the Rotstöckli is a peak within the Urner Alps.\n- It further describes the Urner Alps as part of the Western Alps, a larger mountain range.\n- Therefore, the larger area that contains the region encompassing the Rotstöckli is the Western Alps, as deduced from the hierarchical geographical categorization provided.\n\nAnswer: Western Alps",
    },
]

In [9]:
from itertools import combinations

example_combinations = []
for i in range(1, 3):
    example_combinations.extend(list(combinations(selected_few_shot_examples, i)))

len(example_combinations)

3

In [10]:
results = []

for examples in tqdm(example_combinations):
    example_ids = tuple([example["id"] for example in examples])
    qa_func = partial(answer_question_cte, examples=examples, completion_kwargs={'temperature': 0.1})
    for i in range(1, N_RUNS+1):
        df_cte, scores = benchmark(df, qa_func, perfect_retrieval, ignore_errors=False)
        results.append({**scores, 'examples': example_ids, "run": i})

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
report_df = pd.DataFrame.from_records(results, columns=['examples', 'run', 'exact_match', 'f1'])
agg_report_df = report_df.drop(columns=['run']).groupby(['examples']).agg(['min', 'mean', 'max', 'std'])

In [12]:
from datetime import datetime
suffix = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
report_df.to_json(f'./cte-prompt-optim-results-{suffix}.jsonl', orient='records', lines=True)

In [13]:
agg_report_df.sort_values(('f1', 'mean'), ascending=False)

exact_match                  \
                                                   min      mean   max   
examples                                                                 
(2hop__784447_126070, 2hop__823584_776926)        0.55  0.563333  0.57   
(2hop__823584_776926,)                            0.58  0.583333  0.59   
(2hop__784447_126070,)                            0.51  0.523333  0.53   

                                                            f1            \
                                                 std       min      mean   
examples                                                                   
(2hop__784447_126070, 2hop__823584_776926)  0.011547  0.684868  0.699046   
(2hop__823584_776926,)                      0.005774  0.685320  0.690292   
(2hop__784447_126070,)                      0.011547  0.645732  0.653270   

                                                                
                                                 max       std  
examples                                                        
(2hop__784447_126070, 2hop__823584_776926)  0.717540  0.016758  
(2hop__823584_776926,)                      0.700114  0.008506  
(2hop__784447_126070,)                      0.659651  0.007032

In [14]:
agg_report_df.sort_values(('exact_match', 'mean'), ascending=False)

exact_match                  \
                                                   min      mean   max   
examples                                                                 
(2hop__823584_776926,)                            0.58  0.583333  0.59   
(2hop__784447_126070, 2hop__823584_776926)        0.55  0.563333  0.57   
(2hop__784447_126070,)                            0.51  0.523333  0.53   

                                                            f1            \
                                                 std       min      mean   
examples                                                                   
(2hop__823584_776926,)                      0.005774  0.685320  0.690292   
(2hop__784447_126070, 2hop__823584_776926)  0.011547  0.684868  0.699046   
(2hop__784447_126070,)                      0.011547  0.645732  0.653270   

                                                                
                                                 max       std  
examples                                                        
(2hop__823584_776926,)                      0.700114  0.008506  
(2hop__784447_126070, 2hop__823584_776926)  0.717540  0.016758  
(2hop__784447_126070,)                      0.659651  0.007032

In [15]:
from collections import defaultdict

example_score_mapping = defaultdict(list)

for _, row in report_df.iterrows():
    for example in row['examples']:
        example_score_mapping[example].append(row['f1']) 

sorted([(np.array(scores).mean(), id) for id, scores in example_score_mapping.items()], reverse=True)

[(0.6946691734191733, '2hop__823584_776926'),
 (0.6761584064084064, '2hop__784447_126070')]

In [16]:
selected_ids = [
    "2hop__16777_419765",
    "2hop__823584_776926"
]
with open("selected-cte-few-shot-examples.json", "w") as f:
    json.dump([example for example in few_shot_examples if example["id"] in selected_ids], f, indent=2)

NameError: name 'few_shot_examples' is not defined